In [68]:
import json
import pickle
import numpy as np
from operator import itemgetter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.externals import joblib

In [69]:
# Read files
with open('clean_data_leap/visit_mappings.pkl', 'rb') as f:
        visit_mappings = pickle.load(f)

with open('clean_data_leap/sorted_diag_list.pkl', 'rb') as f:
        sorted_diag_list = pickle.load(f)

with open('clean_data_leap/sorted_drug_list.pkl', 'rb') as f:
        sorted_drug_list = pickle.load(f)

with open('clean_data_leap/diag_order.pkl', 'rb') as f:
        diag_order = pickle.load(f)

In [70]:
# One hot encoding and buid matrix
def feature_matrix(visit_mappings, order):
    one_hot_matrix = list()
    for mapping in visit_mappings:
        matrix_row = [0 for _ in range(2001)]
        for diag in mapping['diag_list']:
            matrix_row[order[diag]] = 1
        one_hot_matrix.append(matrix_row)
    return one_hot_matrix

In [71]:
# One hot encoding and buid vector
def feature_vector(visit_mappings, label):
    one_hot_vec = [0 for _ in range(len(visit_mappings))]
    for i in range(len(one_hot_vec)):
        if label[0] in visit_mappings[i]['drug_list']:
            one_hot_vec[i] = 1
    return one_hot_vec

In [72]:
# Split data by year and generate indices
def split_indices(visit_mappings, year):
    train = list()
    test = list()
    for i in range(len(visit_mappings)):
        if int(visit_mappings[i]['visit_year']) >= int(year):
            test.append(i)
        else:
            train.append(i)
    return train, test

In [73]:
# Find the relationship between medicine and diagnosis by logistic regression
# @return: a list: 0-array:intercept, 1-dict:decisive diags & coef, 2-dict:strong positive diags & coef, 3-dict:negative diags & coefs
#                  4-float:precision, 5-float:recall, 6-float:AUC, 7-lr object:logistic regression model
def logistic_regression(matrix, vec, train, test):

    key_info = list()

    # Split data into training set and test set
    X_train = np.matrix([matrix[i] for i in train])
    X_test = np.matrix([matrix[i] for i in test])
    y_train = [vec[i] for i in train]
    y_test = [vec[i] for i in test]

    # Build logistic model
    lr = LogisticRegression()
    lr.fit(X_train, y_train)

    # Prediction
    y_predict = lr.predict(X_test)

    # Evaluate prediction
    print("\n---------------------Classification Report--------------------\n")
    print(classification_report(y_test, y_predict))
    print("\n-----------------------Confusion Matrix-----------------------\n")
    print(confusion_matrix(y_test, y_predict))
    for ele in y_test:
        if ele == 1:
            auc = roc_auc_score(y_test, y_predict)
            break
        else:
            auc = 0
    print("\n------------------------ROC/AUC SCORE-------------------------\n")
    print("%-10.4f" %(auc))
    key_info.append(precision_score(y_test, y_predict))
    key_info.append(recall_score(y_test, y_predict))
    key_info.append(auc)

    key_info.append(lr)

    return key_info

In [74]:
# Write models
diag_matrix = feature_matrix(visit_mappings, diag_order)
train, test = split_indices(visit_mappings, '2181')
model_index = 0
with open('diag_drug_top538.txt', "w") as f:
    for drug in sorted_drug_list[:538]:
        drug_vector = feature_vector(visit_mappings, drug)
        key_info = logistic_regression(diag_matrix, drug_vector, train, test)

        relation = {}
        relation["drug_name"] = drug[0]
        relation["precision"] = key_info[0]
        relation["recall"] = key_info[1]
        relation["AUC"] = key_info[2]

        lr = key_info[3]
        joblib.dump(lr, "logistic_models_top538/model" + str(model_index) + ".m")
        model_index += 1

        f.write(json.dumps(relation, ensure_ascii=False) + "\n")

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)



---------------------Classification Report--------------------

              precision    recall  f1-score   support

           0       0.75      0.80      0.77      3968
           1       0.73      0.67      0.70      3192

    accuracy                           0.74      7160
   macro avg       0.74      0.73      0.73      7160
weighted avg       0.74      0.74      0.74      7160


-----------------------Confusion Matrix-----------------------

[[3166  802]
 [1063 2129]]

------------------------ROC/AUC SCORE-------------------------

0.7324    


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)



---------------------Classification Report--------------------

              precision    recall  f1-score   support

           0       0.75      0.82      0.79      4214
           1       0.71      0.61      0.66      2946

    accuracy                           0.74      7160
   macro avg       0.73      0.72      0.72      7160
weighted avg       0.73      0.74      0.73      7160


-----------------------Confusion Matrix-----------------------

[[3476  738]
 [1147 1799]]

------------------------ROC/AUC SCORE-------------------------

0.7178    


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


...


---------------------Classification Report--------------------

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7065
           1       0.00      0.00      0.00        95

    accuracy                           0.99      7160
   macro avg       0.49      0.50      0.50      7160
weighted avg       0.97      0.99      0.98      7160


-----------------------Confusion Matrix-----------------------

[[7055   10]
 [  95    0]]

------------------------ROC/AUC SCORE-------------------------

0.4993    


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)



---------------------Classification Report--------------------

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7057
           1       0.48      0.10      0.16       103

    accuracy                           0.99      7160
   macro avg       0.73      0.55      0.58      7160
weighted avg       0.98      0.99      0.98      7160


-----------------------Confusion Matrix-----------------------

[[7046   11]
 [  93   10]]

------------------------ROC/AUC SCORE-------------------------

0.5478    


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)



---------------------Classification Report--------------------

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      7075
           1       0.08      0.01      0.02        85

    accuracy                           0.99      7160
   macro avg       0.54      0.51      0.51      7160
weighted avg       0.98      0.99      0.98      7160


-----------------------Confusion Matrix-----------------------

[[7064   11]
 [  84    1]]

------------------------ROC/AUC SCORE-------------------------

0.5051    


In [75]:
with open('logistic_models_top538/matrix.pkl', 'wb') as f:
        pickle.dump(diag_matrix, f)

with open('logistic_models_top538/train.pkl', 'wb') as f:
        pickle.dump(train, f)

with open('logistic_models_top538/test.pkl', 'wb') as f:
        pickle.dump(test, f)